<a href="https://colab.research.google.com/github/PDNow-Research/PDNow/blob/main/HandPD/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# Data Science
import re
import csv
import json
import itertools
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# General
import os
import sys
import time
import math
import random
from datetime import date
import warnings
current_date = date.today()
warnings.filterwarnings("ignore")

# SVM
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix, average_precision_score, classification_report, accuracy_score
from sklearn.model_selection import StratifiedKFold, KFold


Here's the plan. 

To train the model, we're going to combine a BUNCH of data. Data from Meander && Spiral from Old Hand PD and New Hand PD. Just to see how it goes. Then we'll try more detailed extraction.

Also, try only on new with age?

Run simple SVM based on output of neural network model and age and some other stats :)

CAN POSSIBLY: just extract 5 PD patients and 5 control patients for testing from new hand pd set and just train on everything else. That should be fine...... Then ez to test, just pass those in and evaluate results per patient, perhaps reach 90% accuracy with this 

We also do cross validation/validation set? then.

also, hyperparameter tuning? Can get above 90% accuracy maybe with that!

Discuss more with TT

### Extracting from Old Hand PD

In [4]:
spiral_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/Spiral_HandPD.txt', delimiter=' ', index_col=0, names=['Image', 'Label', 'RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff'])

In [5]:
spiral_df.shape

(368, 10)

In [6]:
spiral_df

,Label,RMS,Max_dist,Min_dist,SD,MRT,Max_ET,Min_ET,SD_ET,HT_ET_Diff
Image,,,,,,,,,,
1,1,3521.258301,6247.052734,30801.99219,0.014133,26.785328,176.600113,0.002130,1781.795898,0.250000
2,1,4098.876465,6032.535156,34369.70313,0.022838,26.529615,168.352737,0.084960,1443.217529,0.273585
3,1,3854.601807,6453.114746,34709.44531,0.000251,23.670755,180.898300,0.009303,1621.750000,0.256329
4,1,4069.221924,6844.231445,32181.26367,0.000168,23.456329,179.116043,0.021419,1454.390137,0.249221
5,1,4104.271973,6949.925293,36444.95313,0.004731,22.488258,188.256210,0.000000,1553.536499,0.214511
...,...,...,...,...,...,...,...,...,...,...
364,2,5593.215820,7997.703613,36371.22656,0.000190,26.964298,179.071930,0.059952,1885.214478,0.198653
365,2,7986.296387,7497.133789,29909.39063,2.613612,22.278151,203.759109,0.033372,1667.547974,0.241379
366,2,4652.542969,7275.474121,39475.41406,0.240567,21.622019,198.719940,0.000000,1469.676147,0.300000


In [7]:
spiral_df['Label'].value_counts()

2    296
1     72
Name: Label, dtype: int64

In [8]:
spiral_df.dtypes

Label           int64
RMS           float64
Max_dist      float64
Min_dist      float64
SD            float64
MRT           float64
Max_ET        float64
Min_ET        float64
SD_ET         float64
HT_ET_Diff    float64
dtype: object

In [9]:
# extracting necessary columns from spiral df
X_spiral = spiral_df[['RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff']]
y_spiral = spiral_df['Label']

y_spiral = pd.DataFrame(y_spiral)

In [10]:
meander_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/Meander_HandPD.txt', delimiter=' ', index_col=0, names=['Image', 'Label', 'RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff'])

In [11]:
meander_df.shape

(368, 10)

In [12]:
spiral_df['Label'].value_counts()

2    296
1     72
Name: Label, dtype: int64

In [13]:
# choosing all patients from meander_df
X_meander = meander_df[['RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff']]

y_meander = meander_df['Label']
y_meander = pd.DataFrame(y_meander)

### Extracting from New HandPD

In [14]:
new_spiral_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/NewSpiral.csv', skiprows = 1, index_col = 0, names=['ID_EXAM', 'Image', 'ID_PATIENT', 'Label', 'GENDER', 'HANDEDNESS', 'AGE', 'RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff']) # skiprows to remove first row of old columns names

In [15]:
new_spiral_df.columns

Index(['Image', 'ID_PATIENT', 'Label', 'GENDER', 'HANDEDNESS', 'AGE', 'RMS',
       'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET',
       'HT_ET_Diff'],
      dtype='object')

In [16]:
# choosing only control from new spiral_df
X_new_spiral = new_spiral_df [new_spiral_df["Label"] == 1]
X_new_spiral = X_new_spiral[['RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff']]

y_new_spiral = new_spiral_df [new_spiral_df["Label"] == 1]
y_new_spiral = y_new_spiral['Label']

y_new_spiral = pd.DataFrame(y_new_spiral)

In [17]:
y_new_spiral.value_counts()

Label
1        140
dtype: int64

In [18]:
new_meander_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/NewMeander.csv', index_col = 0, skiprows =1, names=['ID_EXAM', 'Image', 'ID_PATIENT', 'Label', 'GENDER', 'HANDEDNESS', 'AGE', 'RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff'])

In [19]:
# choosing only control from new meander_df
X_new_meander = new_meander_df [new_meander_df["Label"] == 1]
X_new_meander = X_new_meander[['RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff']]

y_new_meander = new_meander_df [new_meander_df["Label"] == 1]
y_new_meander = y_new_meander['Label']

y_new_meander = pd.DataFrame(y_new_meander)

In [20]:
y_new_meander.value_counts()

Label
1        140
dtype: int64

##**Preprocessing**

###**Normalization**

fi' = (fi - avg)/std

In [21]:
# normalization with the paper's method (formula above)
def feature_normalization(df):
  avg_dev = df.mad(axis = 0)
  std_dev = df.std(axis = 0)

  df = df.sub(avg_dev)
  df = df.divide(std_dev)

  return df

In [22]:
# normalize separately

X_spiral = feature_normalization(X_spiral)
X_new_spiral = feature_normalization(X_new_spiral)
X_meander = feature_normalization(X_meander)
X_new_meander = feature_normalization(X_new_meander)

In [23]:
X = pd.concat([X_spiral, X_new_spiral, X_meander, X_new_meander])
y = pd.concat([y_spiral, y_new_spiral, y_meander, y_new_meander])

In [24]:
X.shape

(1016, 9)

In [25]:
y.shape

(1016, 1)

In [26]:
y.value_counts()

Label
2        592
1        424
dtype: int64

In [28]:
X.head()

,RMS,Max_dist,Min_dist,SD,MRT,Max_ET,Min_ET,SD_ET,HT_ET_Diff
1,3.953296,7.503577,8.383075,-0.287542,9.035704,29.544260,-0.599821,7.554104,4.542268
2,4.729035,7.218892,9.437859,-0.273001,8.942024,28.127675,2.269529,5.968046,5.044539
3,4.400975,7.777041,9.538302,-0.310731,7.894690,30.282525,-0.351338,6.804375,4.677052
4,4.689209,8.296090,8.790853,-0.310870,7.816136,29.976401,0.068378,6.020384,4.525678
5,4.736282,8.436356,10.051400,-0.303248,7.461486,31.546333,-0.673607,6.484831,3.786487


###Train-Test-Split

In [26]:
# Train-Test Split, 80% train, 20% test, stratification across PD diagnosis (label)
y_label = y.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.15, stratify = y_label)


In [27]:
X_train = X_train.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)

In [28]:
X_test = X_test.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [29]:
len(y_train), len(y_test)

(863, 153)

###**Quick Check**

In [30]:
y_train.value_counts()

Label
2        503
1        360
dtype: int64

In [31]:
y_test.value_counts()

Label
2        89
1        64
dtype: int64

In [32]:
# y_val.value_counts()

##**SVM Implementation**

###**Training**

In [57]:
clf = SVC(kernel = 'rbf', probability = True, class_weight = 'balanced', C = 10)
clf.fit(X_train, y_train)

SVC(C=10, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

To note, it may be useful to look at the weights for this model (see which features are most valuable), but these are only relevant for a linear kernel. RBF kernel does not have relevant/interpretable weights. And of course, linear kernel is not very helpful and good in predicting (51% accuracy). The features are not linearly separable.

In [58]:
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)

In [59]:
y_pred = pd.Series(y_pred)

##**Results**

In [60]:
clf.score(X_test, y_test)

0.8888888888888888

In [79]:
threshold_pred = (clf.predict_proba(X_test)[:,1] >= 0.8).astype(int) + 1
threshold_acc = accuracy_score(y_test, threshold_pred)

In [80]:
threshold_acc

0.7647058823529411

In [40]:
y_test.dtypes

Label    int64
dtype: object

In [41]:
target_names = ['Control', 'PD']
results = classification_report(y_test, y_pred, target_names = target_names, output_dict=True)
results = pd.DataFrame(results).transpose()
conf_mat = confusion_matrix(y_test, y_pred)

In [42]:
results

,precision,recall,f1-score,support
Control,1.000000,0.703125,0.825688,64.000000
PD,0.824074,1.000000,0.903553,89.000000
accuracy,0.875817,0.875817,0.875817,0.875817
macro avg,0.912037,0.851562,0.864621,153.000000
weighted avg,0.897664,0.875817,0.870982,153.000000


In [43]:
conf_mat

array([[45, 19],
       [ 0, 89]])

* Consider probability threshold

In [44]:
TN, FP, FN, TP = conf_mat.ravel()

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)

# Specificity or true negative rate
TNR = TN/(TN+FP) 

# Precision or positive predictive value
PPV = TP/(TP+FP)

# Negative predictive value
NPV = TN/(TN+FN)

# Fall out or false positive rate
FPR = FP/(FP+TN)

# False negative rate
FNR = FN/(TP+FN)

# False discovery rate
FDR = FP/(TP+FP)

print("TP: ", TP)
print("TN: ", TN)
print("FP: ", FP)
print("FN: ", FN)

print("Sensitivity: ", TPR)
print("Specificity: ", TNR)
print("NPV: ", NPV)
print("PPV: ", PPV)

TP:  89
TN:  45
FP:  19
FN:  0
Sensitivity:  1.0
Specificity:  0.703125
NPV:  1.0
PPV:  0.8240740740740741


Can we somehow get the weights for the model to learn which features are considered most useful?

## **Log Reg Implementation**

In [45]:
clf = LogisticRegression(penalty='l1', C = 1.0, class_weight=None, random_state = None, solver = 'liblinear')
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
clf.score(X_test, y_test)

0.8562091503267973

In [47]:
target_names = ['Control', 'PD']
results = classification_report(y_test, y_pred, target_names = target_names, output_dict=True)
results = pd.DataFrame(results).transpose()
conf_mat = confusion_matrix(y_test, y_pred)

## **Cross-Validation**

In [49]:
scores_df = pd.DataFrame(columns = ['acc', 'c', 'cw', 'k'])

for i in range(10):
  train_labels = y_train.to_numpy() # train is statified so stratifying on train_labels should be stratified to whole set
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state = i, test_size = 0.1, stratify = train_labels)
  
  X_val = X_val.reset_index(drop = True)
  y_val = y_val.reset_index(drop = True)

  for c in [0.01, 0.1, 1, 10, 100]:
    for cw in ['balanced', None]:
      for k in ['rbf','linear','poly']:
        clf = SVC(kernel = k, probability = True, class_weight = cw, C = c)
        clf.fit(X_train, y_train)

        thresholds = list(np.arange(0, 1, 0.01))
        best_acc = 0.0
        best_threshold = 0.0

        # tuning for best threshold
        for thresh in thresholds:
          y_pred = (clf.predict_proba(X_val)[:,1] >= thresh).astype(int) + 1
          curr_acc = accuracy_score(y_val, y_pred)
          if (curr_acc >= best_acc):
            best_acc = curr_acc
            best_threshold = thresh

        row_df = pd.DataFrame([[best_acc, best_threshold, c, cw, k]], columns = ['acc', 'thresh', 'c', 'cw', 'k'])
    
        scores_df = pd.concat([scores_df, row_df], ignore_index = True, axis=0)
  print (f'fold number {i+1} completed, score number {scores_df.shape[0]}')

fold number 1 completed, score number 30
fold number 2 completed, score number 60
fold number 3 completed, score number 90
fold number 4 completed, score number 120
fold number 5 completed, score number 150
fold number 6 completed, score number 180
fold number 7 completed, score number 210
fold number 8 completed, score number 240
fold number 9 completed, score number 270
fold number 10 completed, score number 300


In [53]:
scores_df['acc'].idxmax()

168

In [56]:
scores_df.loc[168]

acc       0.934783
c               10
cw        balanced
k              rbf
thresh        0.73
Name: 168, dtype: object

In [83]:
best_avg_acc = 0.0
best_avg_hyps = [0.0, 0.0, 0.0]
for c in [0.01, 0.1, 1, 10, 100]:
  for cw in ['balanced', None]:
    for k in ['rbf','linear','poly']:
      filtered = scores_df[(scores_df["cw"] == cw) & (scores_df['c'] == c) & (scores_df["k"] == k)]
      avg_acc = filtered["acc"].mean()
      if (avg_acc >= best_avg_acc):
        best_avg_acc = avg_acc
        best_avg_hyps[0] = c
        best_avg_hyps[1] = cw
        best_avg_hyps[2] = k
print("best_avg_acc: ", best_avg_acc)
print(best_avg_hyps)

best_avg_acc:  0.8704239269793179
[100, 'balanced', 'rbf']


In [84]:
scores_df[(scores_df["cw"] == 'balanced') & (scores_df['c'] == 100) & (scores_df["k"] == 'rbf')]

,acc,c,cw,k,thresh
24,0.884615,100.0,balanced,rbf,0.64
54,0.814286,100.0,balanced,rbf,0.63
84,0.857143,100.0,balanced,rbf,0.67
114,0.842105,100.0,balanced,rbf,0.68
144,0.901961,100.0,balanced,rbf,0.73
174,0.934783,100.0,balanced,rbf,0.38
204,0.880952,100.0,balanced,rbf,0.63
234,0.864865,100.0,balanced,rbf,0.55
264,0.823529,100.0,balanced,rbf,0.68
294,0.900000,100.0,balanced,rbf,0.78


In [85]:
clf = SVC(kernel = 'rbf', probability = True, class_weight = 'balanced', C = 100)
clf.fit(X_train, y_train)

SVC(C=100, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [90]:
thresholds = list(np.arange(0, 1, 0.01))

best_thresh_acc = 0.0
best_thresh = 0.0

for thresh in thresholds:
  threshold_pred = (clf.predict_proba(X_test)[:,1] >= thresh).astype(int) + 1
  threshold_acc = accuracy_score(y_test, threshold_pred)

  if threshold_acc >= best_thresh_acc:
    best_thresh_acc = threshold_acc
    best_thresh = thresh

print(best_thresh, best_thresh_acc)

0.59 0.8888888888888888
